## Import required packages

In [ ]:
import os
from typing import Optional, Tuple
from collections import Counter
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import calendar
import configparser


In [ ]:
# Seting up the configuration file
config = configparser.ConfigParser()
config.read('../cfg/config.ini')

In [ ]:
# Set pandas display options
# Adjust number of decimal displayed
pd.options.display.float_format = '{:.1f}'.format
# Adjust pandas settings to display all rows
pd.set_option('display.max_rows', None)

## First step: Load datasets as Pandas *Dataframes*

Set your working directory to the one containing the datasets to make it easier to load them in memory.

In [ ]:
# Print the current working directory
print('Current working directory: ' + os.getcwd())

In [ ]:
# Change the current working directory if needed
#os.chdir("C:/path/to/new/directory")
os.chdir("/media/data/lvmh")
# Print the new current working directory
print('Current working directory: ' + os.getcwd())

In [ ]:
# Print the files/datasets list in the working directory
print(os.listdir(os.getcwd()))

In [ ]:
# Load the datasets using the read_csv() method. You can specify the separator used in the csv file. By default, the separator is ","
products_df = pd.read_csv('product_inf_2000.csv')
client_df = pd.read_csv('client_inf_2000.csv',sep=';')
transactions_df = pd.read_csv('transac_inf_2000.csv', sep=',')

In [ ]:
products = pd.read_csv('product.csv')
client = pd.read_csv('client.csv', sep=';')
transactions = pd.read_csv('transac.csv', sep=';')

In [ ]:
# Concaténation des DataFrames
combined_products = pd.concat([products, products_df], ignore_index=True)

# Harmonisation des types de colonnes selon le plus grand DataFrame
for col in products_df.columns:
    combined_products[col] = combined_products[col].astype(products_df[col].dtype)

# Vérification des informations du DataFrame combiné
print(combined_products.info())

# Optionnel : sauvegarde en CSV
combined_products.to_csv('combined_products.csv', index=False, sep=',', na_rep='')


In [ ]:
# Concaténation des DataFrames
combined_client = pd.concat([client, client_df], ignore_index=True)

# Harmonisation des types de colonnes selon client_df (le plus grand DataFrame)
for col in client_df.columns:
    combined_client[col] = combined_client[col].astype(client_df[col].dtype)

# Vérification des informations du DataFrame combiné
print(combined_client.info())

# Optionnel : sauvegarde en CSV
combined_client.to_csv('combined_client.csv', index=False, sep=',', na_rep='')


In [ ]:
transactions.info()
transactions_df.info()

In [ ]:
import pandas as pd

# Renommer la colonne 'website_version' en 'country' pour uniformiser
transactions_df = transactions_df.rename(columns={'website_version': 'country'})

# Concaténation
combined_transactions = pd.concat([transactions, transactions_df], ignore_index=True)

# Harmonisation des types selon transactions_df
for col in transactions_df.columns:
    combined_transactions[col] = combined_transactions[col].astype(transactions_df[col].dtype)

# Vérification
print(combined_transactions.info())

# Optionnel : sauvegarde en CSV
combined_transactions.to_csv('combined_transactions.csv', index=False, sep=',', na_rep='')


The head() method on Dataframes displays the top 5 rows. The tail() method displays the last 5 rows. You can tune the number of rows to diplay in parameters, for example try head(10).

In [ ]:
# Display a sample of data with the head() method
combined_products.head()

In [ ]:
# Display a sample of data with the tail() method
combined_transactions.tail(2)

In [ ]:
# Display a sample of data with the tail() method
combined_client.head(3)

The info() method is also useful to have a short report of the dataframe and its data quality:
* number of rows
* column name
* column type
* Number of missing cells of a column

In [ ]:
combined_products.info()

While the `products_df` dataframe describe products (each product_id is called a "SKU" for *Stock Keeping Unit*) and their attributes, `transactions_df` will give very useful details about sales: which product is sold? Which quantity? Which store? When?

In [ ]:
combined_client.head()

The client_df dataframe will give you information on the clients' profiles and purchasing habits. Which kind of clients purchased which products? In what quantity? By which channel? In which country ?...

The Clients Table is particular: each line doesn't correspond to a client, but is at the granularity group of clients x week x product macro family x age x nationality x reachability x is_big_client x channel of purchase x zone of purchase.

For reasons of privacy, we had to regroup the clients in terms of age, etc. The column 'clients' represents the group size. The "items_bought" columns indicate the quantity purchased.

Yet that is not a very convenient table to handle as such. We would prefer that each line corresponds to a client. Therefore, we are going to make an approximation by stating that each line corresponds to a client, whose number of items bought is the one indicated in the table, divided by the cluster size.

That will be an opportunity for us to learn how to build new columns based on other columns.

In [ ]:
## Compute the average number of products purchased per client in each group:
combined_client['items_bought_norm'] = combined_client['items_bought'] / combined_client['clients']

In [ ]:
combined_client['items_bought_norm'].mean()

On average, a client has purchased 1,17 products.

## Data Exploration: Compute basic statistics on data

The describe() method computes basic statistics on numerical columns. On the example below the only numerical column is the price, hence the method computes count, mean, standard deviation, minimum and maximum value and quantiles for the price column.

In [ ]:
# The describe() method computes basic statistics on numerical columns
combined_products.describe()

You can use the nunique() method to get the number of distinct values for all columns.

In [ ]:
combined_transactions.nunique()

You can also apply the same method to a specific column :

In [ ]:
# Get the number of distinct SKUs sold
combined_transactions['product_id'].nunique()

If you're interested in a specific column, you can apply the unique() method at column-level to obtain an array of all the distinct values in this column:

In [ ]:
combined_transactions['product_id'].unique()

## Basic Statistics

Use the mean() method to quickly compute the average over a dataframe's column. You can check the result is the same as the one given by the describe() method above.

In [ ]:
# Get the average price of products
combined_products['price_fr_eur'].mean()

mean() is part of a list of computing methods you can use for any numerical column: mean, sum, max, min...

In [ ]:
# Get the min and max price of products
print(
    'Minimum price of the products dataframe: {}'.format(combined_products['price_fr_eur'].min()),
    'Maximum price of the products dataframe: {}'.format(combined_products['price_fr_eur'].max()),
    sep='\n'
)

Computing statistics is part of data exploration, essential in all data analysis and data science projets. Product, store and transaction data are mandatory for analyzing how products/collections are successful through time and space.

Using value_counts() method, you can have a quick overview of products popularity on the website in a selected country for example. Actually, value_counts() method computes the number of occurrences for each value of a column.

In [ ]:
print(combined_transactions.product_id.nunique(), combined_transactions.product_id.unique(), sep='\n')

Below, the number of occurrences correspond to the number of rows in the dataframe for each country.

In [ ]:
# Top countries in website data
combined_transactions.country.value_counts()

You can use the to_frame() method to get a more readable version of the result displayed:

In [ ]:
# Get the value counts and directly convert to a DataFrame
version_counts_df = combined_transactions['country'].value_counts().to_frame().reset_index()
# Rename columns for clarity
version_counts_df.columns = ['Country', 'Count']
# Display the DataFrame
version_counts_df

How to compute the most represented SKU in transaction data? 

In [ ]:
combined_transactions.product_id.value_counts().head(10)

As you can see above, you can chain multiple methods: here head() follows value_counts().

If you want to look at the data for a specific value of a specific column, below a simple way to do that:

In [ ]:
# Show the shape of web_df dataframe, i.e. show the tuple (number of rows, number of columns)
print(combined_transactions.shape)

# Now show the shape of France rows
print(combined_transactions[combined_transactions.country == 'FRANCE'].shape)

Now let's have a look to the most represented country: China. Are these top products the same as for a specific country?

In [ ]:
# Top 10 SKU with the greatest number of rows in China
combined_transactions[combined_transactions.country == 'CHINA'].product_id.value_counts().head(10)

Is the number of rows the right indicator for defining product populariy? Insights will depend on your ability to identify the right indicators to consider. Rather than using the number of rows, transaction data may have more interesting column like sales, number of clients or number of transactions...


## Join Dataframes

You'll have noticed that some datasets have supplementary information from one another. For example, we would like to add the product's information in the transactions dataset.
To do this, use the merge() method. If you are familiar with SQL the merge() method works similarly as as JOIN between tables. You will need to specify the type of join you want. We typically use left join, meaning we keep all data from the left dataset and will add matching rows from the right dataset. And specify the key for joining, which is the column that is present in both datasets.

For more information about dataframe merging in pandas, this page https://pandas.pydata.org/docs/user_guide/merging.html is a nice reading!

In [ ]:
# Merge transactions and products datasets
transactions_with_product = combined_transactions.merge(combined_products, how='left', left_on='product_id', right_on='product_id')

In the new dataframe transactions_with_product, we will have all the transactions lines with the corresponding products data added in each row.

In [ ]:
# Display a sample of the new merged dataframe
transactions_with_product.head(2)

As precised above, a left join will keep all the data from the left dataframe (here `transactions_df`) while adding the information from the right dataframe. This addition is based on the keys given by `left_on` and `right_on` args. Actually, if the keys have the same value, you can just consider `on` arg:
```python
transactions_with_product = transactions_df.merge(products_df, how='left', on='product_id')

```

It is always a good idea to check that you have not lost or gained any data after a merge! To this end, you can compare the number of rows in your dataframes before and after the merge. They need to be the same.  

In [ ]:
combined_transactions.shape[0]

In [ ]:
transactions_with_product.shape[0]

You could do the same thing with the client_df and product_df dataframe!

## Compute columns

You might want to compute columns. For example, in our new dataset we have for each transaction the number of SKUs sold. And we have added the price of each SKU from the products dataframe. We would like to compute the total corresponding amount for each line.

In [ ]:
# Compute the total sale amount as a new column of the dataframe
transactions_with_product['sale_amount'] = transactions_with_product['product_quantity'] * transactions_with_product['price_fr_eur']

Now our dataframe contains an additional column named "sale_amount" that contains the total sales amount. 

In [ ]:
transactions_with_product['sale_amount'].describe()

In [ ]:
# Display sample data from our dataframe
transactions_with_product.head(2)

## Aggregate Dataframes

Now, let's say we would like to compute the weekly turnover by year on those products.
We would need to aggregate our total sales by week. The first step is to extract the year and week from our dates. Now, look at the week format:

In [ ]:
transactions_with_product['week'][0]

It's a string! We can isolate the year and the week in two separate columns, then convert them to integers with the .astype() method:

In [ ]:
# Split the 'week_column' into 'year' and 'week_number'
transactions_with_product['year'] = transactions_with_product['week'].str[1:5].astype(int)  # Extract the year part
transactions_with_product['week_number'] = transactions_with_product['week'].str[5:].astype(int)  # Extract the week number part

In [ ]:
## Check that both columns are integer with the dtypes method:
transactions_with_product.dtypes

In [ ]:
transactions_with_product.head(2)

Now we want to sum all the sales amounts for each year and week.
If you are familiar with SQL, it is similar to the GROUP BY function.

In [ ]:
## Obtain total sales amount by year and week:
transactions_agg = transactions_with_product[['year','week_number', 'sale_amount']].groupby(by=['year','week_number']).sum()


The sum method added in the end will compute the sum over all numerical columns not used in the "group by". Hence the resulting dataframe will have summed the number of sales amount.

In [ ]:
transactions_agg

You can also directly specify the columns on which performing aggregations, then reset the index of the dataframe:

In [ ]:
minimal_transactions_agg = transactions_with_product[
    ['year','week_number', 'sale_amount']
].groupby(
    by=['year','week_number']
).sum().reset_index(drop=False)

minimal_transactions_agg

Now, you can compare `sale_amount` values for the same week in different years:

In [ ]:
minimal_transactions_agg[minimal_transactions_agg.week_number == 6]

You can sort your aggregated dataframe on `sale_amount`:

In [ ]:
minimal_transactions_agg[minimal_transactions_agg.week_number == 6].sort_values('sale_amount', ascending=False)

In [ ]:
minimal_transactions_agg[minimal_transactions_agg.week_number == 6].sale_amount.max()

We can thus observe an sales amount increase for week number 6 through the last 2 years.

Previously, we saw that value_counts() method allows to compute number of occurrences for column values. Actually, you also do it in a different way with groupby-like aggregations:

In [ ]:
# Top countries in number of products with value_counts
combined_transactions.country.value_counts()

In [ ]:
# Top countries in number of products with groupby and count agg
combined_transactions.groupby('country')[['product_id']].count().sort_values('product_id', ascending=False)

Since value_counts() gives you the number of rows, count() agg gives you the number of cells per column. In this case, it just appears as a more complicated way to do the same thing.

Now, remember when we wanted to compute top SKU for China with a filter? Now with groupby aggregation we can do it for all countries!

In [ ]:
country_sku_occurrences = combined_transactions.groupby(
    ['country', 'product_id']
).count().reset_index(drop=False).rename(
    columns={'week': 'COUNT'}
)[
    ['country', 'product_id', 'COUNT']
]

country_sku_occurrences.head(3)

Do we have the same results as previously?

In [ ]:
combined_transactions[combined_transactions.country == 'CHINA'].product_id.value_counts().head(10)

In [ ]:
country_sku_occurrences[
   country_sku_occurrences.country == 'CHINA'
].sort_values(
    'COUNT', ascending=False
).head(10)

The only (and major) difference is that now, with the groupby method, these occurrences are computed and stored for all countries. With value_counts() method, each call to this method will involve a new compute. If you deal with big data, it may be more comfortable to get already computed and stored results instead of re-compute them.

In [ ]:
country_sku_occurrences[
    country_sku_occurrences.country == 'FRANCE'
].sort_values(
    'COUNT', ascending=False
).head(10)

Let's explore time of execution for both methods:

In [ ]:
%%timeit

combined_transactions[combined_transactions.country == 'CHINA'].product_id.value_counts().head(10)

In [ ]:
%%timeit

country_sku_occurrences[
   country_sku_occurrences.country == 'CHINA'
].sort_values(
    'COUNT', ascending=False
).head(10)

Our new method with groupby-aggregation is far faster than the previous one with value_count().

## Plot Data

It would be nicer to plot these data on a graph to see the evolution of the weekly turnover.
To analyse a weekly turnover, it might be more relevant to narrow it down to a product or a product category.
Let's compute a new aggregation level for a given category (here, we will use the field: macro_family which works like a "category" field).

In [ ]:
# Here are all the product macro_family models
transactions_with_product['macro_family'].unique()

In [ ]:
# This line gives the overall descending ranking of best-selling macro families in sales
transactions_with_product[
    ['macro_family','sale_amount']
].groupby(
    by=['macro_family']
).sum().sort_values(
    by='sale_amount', ascending=False
)

You can also use .loc[] to filter your dataframe. The resulting dataframe will contain only the lines corresponding to your filters. In the example below, the only remaining rows are going to be the ones related to the "CITY BAGS" product family.

In [ ]:
# Let's select the overall best selling macro family = CITY BAGS
transactions_citybags = transactions_with_product.loc[transactions_with_product['macro_family'] == 'CITY BAGS']

In [ ]:
# Let's aggregate this dataframe. You can specify which column you want to sum on:
transactions_citybags_agg = transactions_citybags.groupby(by=['year','week_number'])['sale_amount'].sum()
transactions_citybags_agg.head()

Now, we have in our transactions_citybags_agg dataframe the aggregated sales amounts for the "City Bags" products only.
We can display this data on a chart.  
There are several ways to plot data in Python, we are going to use the pandas native one using the plot() method.  

*Note that here, our dataframe is already indexed by year and week, so you can use it as your X axis with the parameter "use_index=True". If you want to plot againt another variable, you can specify it in the "x=" parameter and get rid of the "use_index=True".*

In [ ]:
# Plot the total sale amount with respect to year-week
transactions_citybags_agg.plot(kind='line', figsize=(15, 6), y='sale_amount', use_index=True)

If we want to make the x axis more clear, we can format the week indicator to display the corresponding month and year. Note how we manage the weeks 53 and 0 which are invalid week numbers for this specific function:

In [ ]:
def get_month_and_year(week_string):
    # Extract the year and week number from the input string
    year_str = week_string[1:5]
    week_num_str = week_string[5:]

    # Convert the year and week number to integers
    year = int(year_str)
    week_num = int(week_num_str)

    # Extract the month and year of every day of the week and chose the average month and year
    months, years = [], []
    try:
            # Handle week 0 (January of the given year)
            if week_num == 0:
                day = datetime.date.fromisocalendar(year, 1, 1)  # First day of the year
                months.append(day.month)
                years.append(day.year)
    
            # Handle valid week numbers (1 to 52, or 53 if valid)
            else:
                for i in range(1, 8):  # Iterate over the days of the week
                    day = datetime.date.fromisocalendar(year, week_num, i)
                    months.append(day.month)
                    years.append(day.year)
    
    except ValueError:
            # Handle invalid week numbers (like non-existent week 53)
            last_day_of_year = datetime.date(year, 12, 31)
            months.append(last_day_of_year.month)
            years.append(last_day_of_year.year)
        
    average_month = Counter(months).most_common(1)[0][0]
    average_year = Counter(years).most_common(1)[0][0]

    # Convert the month to the corresponding label
    month = calendar.month_name[average_month]

    return month[:3] + '-' + str(average_year)

#Example
week_string = "W202148"
print(get_month_and_year(week_string))


Now we can remap the x axys to display the corresponding month and year

In [ ]:
week_sales = transactions_citybags.groupby(by=['week'])['sale_amount'].sum()
week_sales = week_sales.reset_index(drop=False)
week_sales['month_year'] = week_sales['week'].apply(get_month_and_year)

# Convert 'month_year' to date (using the datetime method) for sorting in order to keep a chronological order in your plot
week_sales['month_year_datetime'] = pd.to_datetime(week_sales['month_year'], format='%b-%Y')

## Aggregate to monthly sales:
month_sales = week_sales.groupby(by=['month_year_datetime'])['sale_amount'].sum().reset_index(drop=False)

In [ ]:
## Plot monthly sales
month_sales.plot(kind='line', figsize=(15,6), y='sale_amount', x='month_year_datetime', rot=45)

To understand the sales part of each universe, store_type, gender, it's prefered to use an histogram plot rather than a classic graph, as it greatly improves both explainability and readability.

Besides, we can use the method plt.subplots() in order to show several plots on a unique figure.

In [ ]:
transactions_with_product.head(2)

In [ ]:
universe_sales = transactions_with_product.groupby(by='universe')['sale_amount'].sum()
gender_sales = transactions_with_product.groupby(by='gender')['sale_amount'].sum()
type_sales = transactions_with_product.groupby(by='store_type_label')['sale_amount'].sum()

plots = [universe_sales, gender_sales, type_sales]
fig, axs = plt.subplots(len(plots), 1, figsize=(7, 3*len(plots)))


for i in range(len(plots)):
    axs[i].hist(x=plots[i].index, weights=plots[i], label='Sales')
    axs[i].legend()

# Rotate x-axis labels by 45 degrees
    axs[i].tick_params(axis='x', rotation=45)

plt.tight_layout()  # Adjust layout to prevent label overlap
plt.show()


To view the same repartition but on a specified selection of data we can compute extracts of the main dataframe following the conditions wanted. Here let's compare the sales in a specific category in two different regions of the world.

Wich category should we choose? Let's look at our best-selling macro_families:

In [ ]:
transactions_with_product[
    ['macro_family','sale_amount']
].groupby(
    by=['macro_family']
).sum().sort_values(
    by='sale_amount', ascending=False
)

As the very best-selling category, we will focus on City bags once again. Now, which countries should we compare? 

In [ ]:
transactions_with_product[
    ['country','sale_amount']
].groupby(
    by=['country']
).sum().sort_values(
    by='sale_amount', ascending=False
)

Let's take the top 2: China and Japan

In [ ]:
## Isolate the CITY BAGS  transactions as we did earlier:
transactions_citybags = transactions_with_product.loc[transactions_with_product['macro_family'] == 'CITY BAGS']

# Let's aggregate this dataframe for each week in each region
transactions_citybags_agg = transactions_citybags.groupby(by=['week','country'])['sale_amount'].sum()
week_sales_cb = transactions_citybags_agg.reset_index(drop=False)
week_sales_cb['month_year'] = week_sales_cb['week'].apply(get_month_and_year)

# Convert 'month_year' to datetime for sorting in order to keep a chronological order in your plot
week_sales_cb['month_year_datetime'] = pd.to_datetime(week_sales_cb['month_year'], format='%b-%Y')

## Aggregate to monthly sales:
month_sales_cb = week_sales_cb.groupby(by=['month_year_datetime','country'])['sale_amount'].sum().reset_index(drop=False)

In [ ]:
month_sales_cb.head()

Now we can create a plot for the two countries:

In [ ]:
# We split the dataframe for the two regions
month_sales_cb_cn = month_sales_cb[month_sales_cb['country'] == 'CHINA'].reset_index(drop=True)
month_sales_cb_jp = month_sales_cb[month_sales_cb['country'] == 'JAPAN'].reset_index(drop=True)

month_sales_cb_cn = month_sales_cb_cn.rename(columns={'sale_amount': 'China Sales'})
month_sales_cb_jp = month_sales_cb_jp.rename(columns={'sale_amount': 'Japan Sales'})

fig, ax = plt.subplots(1, 1, figsize=(10,10))

month_sales_cb_cn.plot(kind='line', figsize=(15,6), y='China Sales', x='month_year_datetime', xlabel='date', ax=ax, title='Sales from 2023 to 2024')
month_sales_cb_jp.plot(kind='line', figsize=(15,6), y='Japan Sales', x='month_year_datetime', xlabel='date', ax=ax)
plt.show()

## Models: Kmeans, Linear Regression and Decision Tree

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np


### Kmeans to create a products' segmentation

Clustering is an unsupervised task used in a wide variety of application, including segmentation and analysis. Let's look here how to create a new segmentation for products based on price and popularity: 

In [ ]:
transactions_with_product.head(2)

We will use the kmeans clustering algorithm. You can find documentation on the model here: https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
## Select the column you want to classify the products on, as well as the product unique identifier that we will want to cluster
transactions_with_product_forkmeans = transactions_with_product[['product_id','count_distinct_transaction','price_fr_eur']]

In [ ]:
transactions_with_product_forkmeans.head(2)

Since the values of "price_fr_eur" is so much higher than "count_distinct_transaction", we rework our values so that they are on the same scale. Otherwise, price will overwhelm the number of transaction in terms of importance in the clustering:

In [ ]:
# Selecting the columns for clustering
X = transactions_with_product_forkmeans[['count_distinct_transaction', 'price_fr_eur']]

# We scale the features using StandardScaler()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Kmeans need to be told in how many cluster it will gather the data. Therefore we need to identify the right number of clusters.

#### Find the right number of clusters: the elbow method:

The elbow method uses the Within Cluster Sum of squares to find the optimal number of clusters. Link:https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/

Let us make the plot:

In [ ]:
# Calculating the Within-Cluster-Sum-of-Squares (WCSS) for different numbers of clusters
wcss = []
for k in range(1, 11):  # Testing k values from 1 to 10
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Plotting the elbow curve
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.xticks(range(1, 11))
plt.grid()
plt.show()

As you can see, the inertia drops very quickly as we increase the number of clusters up to 5 then decreases much more slowly as we keep increasing k. The elbow point typically represents a good balance between minimizing WCSS and not overcomplicating the model. So if we did not know better, 5 might be a good choice as a number of clusters.

In [ ]:
# Applying KMeans with 5 clusters
kmeans = KMeans(n_clusters=5, random_state=1234)
transactions_with_product_forkmeans['cluster'] = kmeans.fit_predict(X_scaled)

Now we have obtained a cluster for each product id. 

In [ ]:
transactions_with_product_forkmeans.head()

Now let's look at a our clusters from a statistical point of view:

In [ ]:
# Grouping by cluster and calculating metrics for each field used in clustering
result_cluster = transactions_with_product_forkmeans.groupby('cluster').agg(
    distinct_product_id=('product_id', 'nunique'), ## Number of product in the cluster
    mean_count_distinct_transaction=('count_distinct_transaction', 'mean'), # Average number of transaction per product in the cluster
    mean_price_fr_eur=('price_fr_eur', 'mean') # Average price per product in the cluster
).reset_index()

In [ ]:
result_cluster

Let's analyse the results:

As you can see, we have 10 products with a very high number of transactions (compared to other clusters) in cluster 3, therefore a cluster made of very popular products. Cluster nbr 3 could be qualified as a "bestsellers" cluster. 

Then, we have a cluster with, on average, more expensive products even though they are less often sold: cluster 2. Therefore, cluster 2 could be "high-end" products cluster, with expensive and more exclusive products.

What could be the name of the other clusters and how could you use them?

### Linear Regression to predict future sales

A linear model makes a prediction by simply computing a weighted sum of the input features plus a constant called the bias term. Here are more information on linear regression: https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LinearRegression.html.
We could use it to try and predict future sales amount:

In [ ]:
month_sales.head()

First we will need to convert the year_month column into numerical values (e.g., ordinal values or integers representing time).

In [ ]:
month_sales["time_index"] = np.arange(len(month_sales))  # Add time index for linear regression


In [ ]:
# Prepare the data
X = month_sales[["time_index"]]  # Independent variable (time)
y = month_sales["sale_amount"]  # Dependent variable (sales)

# Train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict for all historical data points
month_sales["Predicted"] = model.predict(X)

# Predict for future data points. Here we can choose the number of months we want to predict:
future_periods = 6  #Number of months to predict
last_time_index = month_sales["time_index"].iloc[-1]
future_time_index = np.arange(last_time_index + 1, last_time_index + 1 + future_periods).reshape(-1, 1)
future_forecast = model.predict(future_time_index)

# Create a DataFrame for the forecast
forecast_index = pd.date_range(month_sales["month_year_datetime"].iloc[-1] + pd.offsets.MonthBegin(), periods=future_periods, freq="ME")
forecast_month_sales = pd.DataFrame({"Predicted": future_forecast}, index=forecast_index)

# Plot the results
plt.plot(month_sales["month_year_datetime"], month_sales["sale_amount"], label="Actual")
plt.plot(month_sales["month_year_datetime"], month_sales["Predicted"], label="Prediction (Historical)", linestyle="--", color="orange")
plt.plot(forecast_month_sales.index, forecast_month_sales["Predicted"], label="Prediction (Future)", linestyle="--", color="red")
plt.xlabel("Year-Month")
plt.ylabel("Sales Amount")
plt.legend()
plt.show()


Here we have a prediction for the next six months but we can see that the prediction for the historical data does not fit the actual sales very well...  How could you improve it?

### Decision Tree to find clients' preference

Our analysis thus far seems to indicate that CITY BAGS are very popular products, purchased by a wide amount of clients. But what about a family of products that is less popular, like TRAVEL? Could we identify which clients would be more amenable to buying travel products? And what their characteristics are?  

Let's see if we can see which kind of clients' prefer TRAVEL by using a Decision Tree classifier.

Decision Trees are versatile Machine Learning algorithms that can perform both classification and regression tasks, and are capable of fitting complex datasets. Here you can find the documentation: https://scikit-learn.org/1.5/modules/tree.html 

One of the many qualities of Decision Trees is that they require very little data preparation. In fact they don't require feature scaling or centering at all.

In our use case, we are going to use purchases in other categories to try and find a behavioral patterns (that can be used by our business teams) that might predict clients that would enjoy TRAVEL products. Therefore, for this example, all of our data will be numerical.

#### Create dataframe

First we will create a dataframe with each column representing a category of purchase by using the pivot_table method. Link to doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html

In [ ]:
## Prepare the data: keep the columns on which you  would like the classification to happen, as well as the target:

client_df_for_pred =pd.pivot_table(combined_client, values='items_bought', index=['nationality', 'gender','is_reachable','is_big_client','store_zone'],
                       columns=['macro_family'], aggfunc="sum", fill_value=0) ## The fill_value will input 0 if there have been no sales

In [ ]:
client_df_for_pred.head()

##### Build Decision Tree:

First we need to build our target. This will be a binary exercise: has a client purchased a travel product or not?

In [ ]:
## First step: build your target and your features
# Create target (y)
y = (client_df_for_pred['TRAVEL'] > 0).astype(int)  # Binary target: 1 if purchased, 0 otherwise

## Identify the columns that will be used as features (all product categories, excluding the index and 'TRAVEL' column)
X = client_df_for_pred.drop(columns=['TRAVEL'])


In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

The accuracy of 0.82 seems pretty good at first glance but let us look a little deeper with the confusion matrix.

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display the confusion matrix
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Purchase", "Purchase"], yticklabels=["No Purchase", "Purchase"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

The results are pretty encouraging ! The Decision Tree seems to be especially good at identifying which clients will not purchase TRAVEL products, with only 12 clients that were predicted as purchasing when they in fact, did not purchase TRAVEL.  

Let us take a look at the decision tree!

In [ ]:
## Import necessary packages
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Plot the Decision Tree
plt.figure(figsize=(40, 50))  # Adjust figure size for better readability
plot_tree(
    clf, 
    feature_names=X.columns,  # Use feature names for clarity
    class_names=["No Purchase", "Purchase"],  # Label classes
    filled=True,  # Color nodes based on class
    rounded=True,  # Round node corners for aesthetics
    fontsize=12  # Adjust font size
)
plt.title("Decision Tree Visualization")
plt.show()

This Decision Tree gives good results but seems overly complicated with often samples of the population that are below 5 before classifying the sample. It seems very hard to identify generalized rules that we could use in a business setting to identify TRAVEL clients.


Decsion Trees make very few assumptions about the training data, they are nonparametric models so the model structure is free to stick closely to the data without constraints, which can lead to overcomplicating and overfitting. To avoid that we need to restrict the Decision Tree's freedom during training.

There are several aspects that you can tweak in order to improve the tree and make it usable in a business setting.
- The depth of the tree
- Should you use gini impurity or entropy?
- Tune the hyperparameters: min_samples_split, max_leaf_nodes...

Can you improve the identification of TRAVEL clients so that it can be used by the business team?

In [ ]:
import pandas as pd
import re

# Fonction pour convertir 'week' au format correct
def convert_week_format(week_str):
    if pd.isna(week_str):
        return pd.NaT  # Gestion des NaN
    match = re.search(r'W?(\d{4})(\d{2})', week_str)  # Capture YYYY et WW
    if match:
        year, week = match.groups()
        return pd.to_datetime(f'{year}-{week}-1', format='%Y-%W-%w')  # Convertir en lundi de la semaine
    return pd.NaT  # Si erreur, on met NaT

# Fusionner transactions et produits
transactions_with_product = combined_transactions.merge(combined_products, how='left', on='product_id')

# Convertir la colonne 'week' en format datetime
transactions_with_product['week'] = transactions_with_product['week'].apply(convert_week_format)

# Agréger les ventes par semaine et produit
df_sales = transactions_with_product.groupby(['week', 'product_id']).agg({
    'product_quantity': 'sum',
    'price_fr_eur': 'mean',  # Moyenne du prix
    'macro_family': 'first',  # Prendre une seule valeur
    'store_type_label': 'first'
}).reset_index()

# Remplacement des NaN
df_sales.fillna({'price_fr_eur': df_sales['price_fr_eur'].median(), 'macro_family': 'Unknown'}, inplace=True)

print(df_sales.head())  # Vérification des données


EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

#  Distribution des ventes
plt.figure(figsize=(10, 5))
sns.histplot(df_sales['product_quantity'], bins=50, kde=True)
plt.title("Distribution des quantités vendues")
plt.xlabel("Quantité vendue")
plt.ylabel("Fréquence")
plt.show()

#  Analyse des tendances temporelles
df_time_series = df_sales.groupby('week')['product_quantity'].sum()

plt.figure(figsize=(12, 5))
plt.plot(df_time_series, marker='o', linestyle='-')
plt.title("Tendance des ventes au fil du temps")
plt.xlabel("Temps (semaine)")
plt.ylabel("Total des ventes")
plt.grid()
plt.show()

#  Décomposition de la série temporelle (trend, saisonnalité, résidu)
decomp = sm.tsa.seasonal_decompose(df_time_series, model='additive', period=52)  # 52 semaines = 1 an
decomp.plot()
plt.show()

plt.figure(figsize=(10, 5))
sns.scatterplot(x=df_sales['price_fr_eur'], y=df_sales['product_quantity'])
plt.title("Impact du prix sur les ventes")
plt.xlabel("Prix (€)")
plt.ylabel("Quantité vendue")
plt.show()


Modélisation

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error

#  Entraînement du modèle ARIMA
train_size = int(len(df_time_series) * 0.8)
train, test = df_time_series[:train_size], df_time_series[train_size:]

model = ARIMA(train, order=(5,1,0))  # AR(5), I(1), MA(0)
model_fit = model.fit()

#  Prédictions
predictions = model_fit.forecast(steps=len(test))

#  Évaluation des erreurs
mape = mean_absolute_percentage_error(test, predictions)
print(f"MAPE du modèle ARIMA : {mape:.2%}")

plt.figure(figsize=(12, 5))
plt.plot(train, label="Train")
plt.plot(test, label="Test", color='orange')
plt.plot(test.index, predictions, label="Prédictions", color='red', linestyle="dashed")
plt.legend()
plt.title("Prédictions ARIMA vs Réel")
plt.show()


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

#  Correction des erreurs de datetime
df_sales['week_num'] = df_sales['week'].dt.isocalendar().week  # Remplace .dt.week
df_sales['year'] = df_sales['week'].dt.year

#  Définition des features et target
features = ['week_num', 'year', 'price_fr_eur']  
target = 'product_quantity'

X = df_sales[features]
y = df_sales[target]

#  Séparation Train / Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  Entraînement du modèle XGBoost
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model.fit(X_train, y_train)

#  Prédictions
y_pred = xgb_model.predict(X_test)

#  Évaluation
mape_xgb = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE du modèle XGBoost : {mape_xgb:.2%}")

plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r')  # Ligne parfaite
plt.xlabel("Ventes réelles")
plt.ylabel("Prédictions")
plt.title("Performance du modèle XGBoost")
plt.show()


Evaluation

In [ ]:
print(f"MAPE ARIMA : {mape:.2%}")
print(f"MAPE XGBoost : {mape_xgb:.2%}")


Plot pour les Client

In [ ]:
#  Vérification des données
print(combined_client.info())
print(combined_client.describe())

#  Distribution des clients par nationalité
plt.figure(figsize=(12, 5))
sns.countplot(y=combined_client['nationality'], order=combined_client['nationality'].value_counts().index)
plt.title("Répartition des clients par nationalité")
plt.show()

#  Répartition des clients par genre
plt.figure(figsize=(6, 4))
sns.countplot(x=combined_client['gender'])
plt.title("Répartition des clients par genre")
plt.show()

#  Répartition des clients par tranche d'âge
plt.figure(figsize=(8, 5))
sns.countplot(x=combined_client['age'], order=sorted(combined_client['age'].unique()))
plt.title("Distribution des clients par tranche d'âge")
plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler

#  Sélection des features utiles
features = ['clients', 'items_bought', 'is_reachable', 'is_big_client']

#  Normalisation des données
scaler = StandardScaler()
client_scaled = scaler.fit_transform(combined_client[features])

#  Conversion en DataFrame
client_scaled_df = pd.DataFrame(client_scaled, columns=features)


In [ ]:
from sklearn.cluster import KMeans
#  Test de plusieurs nombres de clusters
inertia = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(client_scaled)
    inertia.append(kmeans.inertia_)

#  Tracé de la méthode du coude
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker='o')
plt.title("Méthode du coude : choix du nombre de clusters")
plt.xlabel("Nombre de clusters K")
plt.ylabel("Inertie")
plt.show()


In [ ]:
#  Choix du nombre optimal de clusters (ex : K=4)
optimal_k = 4  
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
combined_client['cluster'] = kmeans.fit_predict(client_scaled)

#  Visualisation des clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=combined_client['clients'], y=combined_client['items_bought'], hue=combined_client['cluster'], palette="viridis")
plt.title("Segmentation des clients")
plt.xlabel("Nombre de clients")
plt.ylabel("Nombre d'articles achetés")
plt.legend(title="Cluster")
plt.show()


In [ ]:
#  Moyenne des variables par cluster
cluster_summary = combined_client.groupby("cluster")[features].mean()
print(cluster_summary)

#  Répartition des clients par segment
plt.figure(figsize=(8, 5))
sns.countplot(x=combined_client['cluster'])
plt.title("Répartition des clients par cluster")
plt.show()


In [ ]:
top_products = combined_transactions.groupby('product_id')['product_quantity'].sum().sort_values(ascending=False).head(10)
print(top_products)


In [ ]:
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Analyse comparative des ventes Web vs Store Physique

In [ ]:
sales_by_store = combined_transactions.groupby(['week', 'store_type_label'])['product_quantity'].sum().reset_index()
plt.figure(figsize=(14, 6))
sns.lineplot(data=sales_by_store, x='week', y='product_quantity', hue='store_type_label')
plt.title('Évolution des ventes Web vs Magasins Physiques')
plt.xlabel('Semaine')
plt.ylabel('Quantité de produits vendus')
plt.show()

Segmentation clients (KMeans) avec variables clés


In [ ]:
client_features = combined_client[['clients', 'items_bought', 'is_big_client', 'is_reachable']]
kmeans = KMeans(n_clusters=4, random_state=42)
combined_client['cluster'] = kmeans.fit_predict(client_features)
sns.countplot(x='cluster', hue='store_type_label', data=combined_client)
plt.title('Segmentation clients selon le type de magasin')
plt.show()

Analyse de l’appétence produit (Produits les plus vendus Web vs Physique)

In [ ]:
product_sales = combined_transactions.groupby(['product_id', 'store_type_label'])['product_quantity'].sum().reset_index()
merged_sales = product_sales.merge(combined_products, on='product_id', how='left')
top_web_products = merged_sales[merged_sales['store_type_label'] == 'Web'].sort_values(by='product_quantity', ascending=False).head(10)
top_store_products = merged_sales[merged_sales['store_type_label'] == 'Physical'].sort_values(by='product_quantity', ascending=False).head(10)
print("Top 10 produits Web:")
print(top_web_products[['product_id', 'sku_description', 'product_quantity']])
print("\nTop 10 produits Magasin:")
print(top_store_products[['product_id', 'sku_description', 'product_quantity']])

Analyse omnicanal (Cross-shop clients entre Web & Store)

In [ ]:
client_cross_shop = combined_client.groupby(['store_type_label', 'is_big_client'])['clients'].sum().reset_index()
sns.barplot(data=client_cross_shop, x='store_type_label', y='clients', hue='is_big_client')
plt.title('Analyse des clients multi-canaux')
plt.ylabel('Nombre de clients')
plt.show()